# Paternity arrays

Paternity arrays are the what sibship clustering is built on in FAPS. They contain information about the probability that each candidate male is the father of each individual offspring. This information is stored in a `paternityArray` object, along with other related information. A `paternityArray` can either be imported directly, or created from genotype data.

## Creating a `paternityArray` from genotype data

To create a `paternityArray` from genotype data we need to specficy `genotypeArray`s for the offspring, mothers and candidate males. Currently only biallelic SNP data are supported.

We will illustrate this with a small simulated example again with four adults and six offspring.

In [1]:
from faps import *
import numpy as np

np.random.seed(49) # this ensures you get exactly the same answers as I do.
allele_freqs = np.random.uniform(0.3,0.5,10)
mypop        = make_parents(4, allele_freqs, family_name='my_population')
progeny      = make_sibships(mypop, 0, [1,2], 3, 'myprogeny')

We need to supply a `genotypeArray` for the mothers. This needs to have an entry for for every offspring, i.e. six replicates of the mother.

In [2]:
mum_index = progeny.parent_index('mother', mypop.names) # positions in the mothers in the array of adults
mothers   = mypop.subset(mum_index) # genotypeArray of the mothers

To create the `paternityArray` we also need to supply information on the genotyping error rate (mu), and population allele frequencies. For the latter, in this case we can either take the population allele frequencies defined above, or estimate them from the data, which will give slightly different answers. The function `paternity_array` creates an object of class `paternityArray`.

In [3]:
mu = 0
sample_af = mypop.allele_freqs()
patlik = paternity_array(offspring = progeny, mothers = mothers, males= mypop, allele_freqs = sample_af, mu=mu)

## `paternityArray` structure

A `paternityArray` inherits information about individuals from found in a `genotypeArray`, for example:

In [12]:
print patlik.candidates
print patlik.mothers
print patlik.offspring

['my_population_0', 'my_population_1', 'my_population_2', 'my_population_3']
['my_population_0', 'my_population_0', 'my_population_0', 'my_population_0', 'my_population_0', 'my_population_0']
['myprogeny_0', 'myprogeny_1', 'myprogeny_2', 'myprogeny_3', 'myprogeny_4', 'myprogeny_5']


The most important part of the `paternityArray` is the likelihood array, which represent the log likelihood that each candidate male is the true father of each offspring individual. In this case it will be a 6x4 dimensional array, which we could view with:

In [5]:
patlik.likelihood

array([[-51.02425987, -42.17216734, -57.43110392, -47.75398282],
       [-51.02425987, -42.17216734, -46.0440735 , -42.17216734],
       [-51.71441154, -42.17440974, -58.12125559, -48.44413449],
       [-56.82947481, -42.86007661, -42.85858896, -43.54798588],
       [-47.97965161, -59.63719565, -43.55546784, -49.4940771 ],
       [-45.9091966 , -53.00543756, -41.48501283, -47.42362208]])

You can see that individuals one (the mother) and four are incompatible as fathers with any offspring, because the first and last columns all show log likelihoods of negative infinity. The second candidate is the only compatible father for the first full-sibship, and the third father is the only compatible father for the second sibship (second and third columns respectively).

In toy example we know the error rate to be zero, and have given it as such. In reality this will almost never be true, and we would usually need to incorporate an error rate. Here's what happens to `patlik` if we use a realistic error rate.

In [5]:
mu = 0.0015
patlik = paternity_array(offspring = progeny, mothers = mothers, males= mypop, allele_freqs = sample_af, mu=mu)
patlik.likelihood

array([[-51.02425987, -42.17216734, -57.43110392, -47.75398282],
       [-51.02425987, -42.17216734, -46.0440735 , -42.17216734],
       [-51.71441154, -42.17440974, -58.12125559, -48.44413449],
       [-56.82947481, -42.86007661, -42.85858896, -43.54798588],
       [-47.97965161, -59.63719565, -43.55546784, -49.4940771 ],
       [-45.9091966 , -53.00543756, -41.48501283, -47.42362208]])

No candidate can now be excluded explicitly, but we can see that the true sires have log liklihoods around ten units higher than the imposters. This means that the true sire is around 22,000-fold more likely to be the true father than the other candidates. That's quite a lot!

The `paternityArray` also includes information that the true sire is not in the sample of candidate males. In this case this is not helpful, because we know sampling is complete, but in real examples is seldom the case. By default this is defined as the liklihood of generating the offspring genotypes given the known mothers genotype and alleles drawn from population allele frequencies. Here, values for the six offspring are higher than the likelihoods for the non-sires, indicating that they are no more likely to be the true sire than a random unrelated individual.

In [7]:
patlik.lik_absent

array([-46.28363291, -48.77700054, -47.36620112, -48.39791318,
       -49.50267811, -47.71331648])

This becomes more informative when we combine likelihoods about sampled and unsampled fathers. For fractional analyses we really want to know the probability that the father was unsampled vs sampled, and how probable it is that a single candidate is the true sire. To do this, we concatenate the values in `patlik.lik_absent` as an extra column to `patlik.likelihoods`, then normalise the array so that values in each row sum to one. It is straightforward to do this:

In [14]:
patprob = patlik.probability()
print patprob.shape

(6, 5)


Printing the shape of the array demonstrates that we have gained a column. If we sum the rows we see that they do indeed add up to one now. Probabilities are stored as log probabilities, so we have to exponentiate first.

In [17]:
np.exp(patprob).sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.])

You can specify the proportion $\theta$ of the population of candidate males which are missing. The likelihoods for non-sampled parents will be weighted by $\theta$, and likelihoods for sampled candidates by $1-\theta$.

In [26]:
patprob2 = patlik.probability(missing_parents=0.1)

Often the mother is included in the sample of candidate males, either because you are using the same array for multiple families, or self-fertilisation is a biological possibility. In a lot of cases though the mother cannot simultaneously be the sperm/pollen donor, and it is necessary to set the rate of self-fertilisation to zero.

In [5]:
patlik.probability(selfing_rate=0)

array([[            -inf,  -1.99493854e-02,  -1.52788860e+01,
         -5.60176487e+00,  -4.13141495e+00],
       [            -inf,  -7.04172407e-01,  -4.57607856e+00,
         -7.04172407e-01,  -7.30900561e+00],
       [            -inf,  -7.42725188e-03,  -1.59542731e+01,
         -6.27715200e+00,  -5.19921863e+00],
       [            -inf,  -9.19505403e-01,  -9.18017758e-01,
         -1.60741467e+00,  -6.45734198e+00],
       [            -inf,  -1.60869630e+01,  -5.23518911e-03,
         -5.94384444e+00,  -5.95244546e+00],
       [            -inf,  -1.15250325e+01,  -4.60778144e-03,
         -5.94321704e+00,  -6.23291143e+00]])

The likelihoods for the mother have changed to 0 (negative infinity on the log scale). You can set any selfing rate between zero and one if you have a good idea of what the value should be and how much it varies. Otherwise it may be better to estimate the selfing rate from the data, or else estimate it some other way.

## Importing a `paternityArray`

Frequently you may wish to save an array and reload it. Otherwise, you may be working with a more exotic system than FAPS currently supports, such as microsatellite markers or a funky ploidy system. In this case you can create your own matrix of paternity likelihoods and import this directly as a `paternityArray`. Firstly, we can save the array we made before to disk by supplying a path to save to:

In [6]:
patlik.write('./data_files/mypatlik.csv')

We can reimport it again using `read_paternity_array`. This function is similar to the function for importing a `genotypeArray`, and the data need to have a specific structure:

1. Offspring names should be given in the first column
2. Names of the mothers are usually given in the second column.
3. If known for some reason, names of fathers can be given as well.
4. Likelihood information should be given *to the right* of columns indicating individual or parental names, with candidates' names in the column headers.
5. The final column should specify a likelihood that the true sire of an individual has *not* been sampled. Usually this is given as the likelihood of drawing the paternal alleles from population allele frequencies.

In [2]:
patlik = read_paternity_array('./data_files/mypatlik.csv', mothers_col=1, likelihood_col=2)